# Train/Test/Validation Splitting

This notebook was used to split the data for training, validation, and testing.

In [2]:
import sys; sys.path.append("..")

import pandas as pd
import plotly.express as px

import Bio.PDB.PDBParser
from Bio.PDB.Polypeptide import protein_letters_3to1

import gvpgnn.paths as paths
import gvpgnn.datamodels as dm

/var/folders/0z/fnhwmrr91f362mk6_bndszhw0000gn/T/ipykernel_33742/731750062.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Open the training data sequences and structure
df = pd.read_csv('../data/cath_w_seqs_share.csv', index_col=0)

## What does the distribution of superfamilies look like?

In [ ]:
# Analyze the number of shared homologous superfamilies.
n_sf_unique = len(df.superfamily.unique())
print(f"In training set of size {len(df)}, there are {n_sf_unique} unique homologous superfamilies")

df[df.superfamily == 1]

In [ ]:
df_sf_counts = df.groupby(by="superfamily").size()

df_sf_counts.index = df_sf_counts.index.astype("str") # convert to categorical index
df_sf_counts.name = "members"

# Show as a column chart:
# fig = px.bar(df_sf_counts, title="Members of each homologous superfamily in the dataset")
# fig.show()

# Show as a pie chart:
fig = px.pie(df_sf_counts.to_frame(), values="members", title="Members of each homologous superfamily in the dataset")
fig.show()

## Dataset Splitting Strategy

- We want to avoid splitting homologous superfamilies across the training and test splits, since this would allow the model to train on examples that are closely related to those in the test set.
- The simplest solution I can think of is to choose superfamilies to put in the test set, and not let the model see any examples from those families in the training set.
- The question is whether we want more diversity in the training set or test set? There are a few huge superfamilies, like `10` which is the largest (about `28.7%` of examples belong to it). So whichever split has those huge families will get a bit less variety.
- My initial decision is to fill up the test set with small superfamilies to maximize the diversity there. We want to see how the model generalizes to many different unseen superfamilies, and I'm guessing that's how this work task will be evaluated.

In [ ]:
fraction_test = 0.2 # of the entire set
fraction_val = 0.2 # of the REMAINING training set

n_test = round(len(df) * fraction_test)
n_training = len(df) - n_test
n_val = round(n_training * fraction_val)
n_train = n_training - n_val

print("\n-- NOMINAL SPLITS:")
print("Dataset size:", len(df))
print("test:", n_test)
print("train:", n_train)
print("val:", n_val)

sf_count_low_to_high: list[tuple[int, int]] = \
  sorted(df.groupby(by="superfamily").size().to_dict().items(), key=lambda x: x[1])

split_superfamilies = dict(train=[], val=[], test=[])
split_counts = dict(train=0, val=0, test=0)

idx = 0

# Build the test set first so that it receives many small superfamilies.
while split_counts["test"] < n_test:
  sf, count = sf_count_low_to_high[idx]
  split_superfamilies["test"].append(sf)
  split_counts["test"] += count
  idx += 1

# Then build the validation set, which we also want to have a diverse set of families.
while split_counts["val"] < n_val:
  sf, count = sf_count_low_to_high[idx]
  split_superfamilies["val"].append(sf)
  split_counts["val"] += count
  idx += 1

# Every remaining example goes in the training set.
split_superfamilies["train"].extend([
  sf_count_low_to_high[i][0] for i in range(idx, len(sf_count_low_to_high))
])

split_counts["train"] = sum([sf_count_low_to_high[i][1] for i in range(idx, len(sf_count_low_to_high))])

print("\n-- ACTUAL SPLITS:")
print("test:", split_counts["test"])
print("val:", split_counts["val"])
print("train:", split_counts["train"])

print("(Note: Due to the size of superfamilies, the actual splits may be slightly different in size.)")

# Make sure that every example is accounted for!
assert(sum(split_counts.values()) == len(df))

# Write the splits to files:
for split_name in ("train", "val", "test"):
  sf_list = split_superfamilies[split_name]
  df_split = df[df.superfamily.isin(sf_list)]
  df_split.to_csv(paths.data_folder(f"{split_name}_cath_w_seqs_share.csv"), index=False)

In [ ]:
# Ensure that no examples are shared across splits!
cath_ids = dict(train=set(), val=set(), test=set())

for split_name in cath_ids:
  df_split = pd.read_csv(paths.data_folder(f"{split_name}_cath_w_seqs_share.csv"))
  cath_ids[split_name] = set(df_split.cath_id.unique())

# Just to be really sure...
assert(cath_ids["train"].isdisjoint(cath_ids["test"]))
assert(cath_ids["train"].isdisjoint(cath_ids["val"]))
assert(cath_ids["val"].isdisjoint(cath_ids["test"]))

In [1]:
architecture_names = {(1, 10): "Mainly Alpha: Orthogonal Bundle",
                      (1, 20): "Mainly Alpha: Up-down Bundle",
                      (2, 30): "Mainly Beta: Roll",
                      (2, 40): "Mainly Beta: Beta Barrel",
                      (2, 60): "Mainly Beta: Sandwich",
                      (3, 10): "Alpha Beta: Roll",
                      (3, 20): "Alpha Beta: Alpha-Beta Barrel",
                      (3, 30): "Alpha Beta: 2-Layer Sandwich",
                      (3, 40): "Alpha Beta: 3-Layer(aba) Sandwich",
                      (3, 90): "Alpha Beta: Alpha-Beta Complex"}

In [4]:
df

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]"
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]"
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]"
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]"
...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]"
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]"
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]"
